# Model Zoo Notebook

This notebook demonstrates how to load a COCO-pretrained YOLO-NAS S model and a hagRID-pretrained YOLO 11 model, and export them to ONNX format using PyTorch, SuperGradients, and Ultralytics.

**Important:**  
This notebook must be executed first before running any other notebooks in this project. It prepares the modela and exports them for further use.

## How to Use

1. **Build and start the Docker Compose environment** as described in the project documentation.
2. **Access this notebook** in your browser at:  
    [http://127.0.0.1:8889/notebooks/model_zoo.ipynb](http://127.0.0.1:8889/notebooks/model_zoo.ipynb)
3. **Run all cells** in order to prepare the models for downstream tasks.

Make sure to follow these steps to ensure the environment and model are set up correctly.

In [1]:
# Import necessary libraries.
from typing import List

import onnx
import os
import torch

from onnx import helper, TensorProto
from super_gradients.common.object_names import Models
from super_gradients.training import models
from ultralytics import YOLO

[2025-05-31 09:22:36] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


## Exporting YOLO-NAS S Model to ONNX

The following code cell loads a COCO-pretrained YOLO-NAS S model using SuperGradients, prepares it for ONNX export, and saves the exported model to the `./models/yolo_nas_s.onnx` path. The process includes:

- Loading the pretrained model and setting it to evaluation mode.
- Preparing the model for conversion with a specified input size.
- Creating a dummy input tensor to simulate a real input.
- Defining input and output names for the ONNX graph.
- Exporting the model to ONNX format using `torch.onnx.export`.

This ONNX model can then be used for inference in other frameworks or deployment environments that support ONNX.

In [2]:
# Load a COCO-pretrained YOLO-NAS S model.
model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
model.eval()

# Prepare the model for ONNX conversion.
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])

# Define a dummy input tensor with the expected shape.
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")

# Specify the input and output names for the ONNX model.
input_names = ["input"]
output_names = ["output_bboxes", "output_classes"]

# Export the model to ONNX format.
torch.onnx.export(
    model,
    dummy_input,
    "/models/yolo_nas_s.onnx",
    input_names=input_names,
    output_names=output_names,
    opset_version=11
)

# Validate the ONNX model.
model = onnx.load("/models/yolo_nas_s.onnx")
onnx.checker.check_model(model)
print("Model is valid!")

[2025-05-31 09:22:38] WARNING - checkpoint_utils.py - :warning: The pre-trained models provided by SuperGradients may have their own licenses or terms and conditions derived from the dataset used for pre-training.
 It is your responsibility to determine whether you have permission to use the models for your use case.
 The model you have requested was pre-trained on the coco dataset, published under the following terms: https://cocodataset.org/#termsofuse
[2025-05-31 09:22:38] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2025-05-31 09:22:38] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


Model is valid!


## ONNX Model Graph Transformation Utility

The following code cell defines a utility function, `transform_io_and_prune_node`, which modifies an ONNX model's computation graph. This function enables advanced manipulation of ONNX models, including:

- Renaming the model's input tensor.
- Removing all existing outputs and creating new output chains using Transpose and Identity nodes.
- Assigning new names to the outputs and connecting them to specified internal tensors.
- Removing a specific node (e.g., a Concat node) from the model by its name.

This utility is useful for customizing ONNX models for downstream tasks, such as adapting the model's input/output interface or pruning unnecessary nodes before deployment. The function takes paths to the input and output ONNX models, new input/output names, internal tensor names for output chaining, and the name of the node to remove.

In [3]:
def transform_io_and_prune_node(
    model_path: str, output_path: str,
    new_input_name: str,
    new_output_names: List,
    internal_tensor_names: List,
    remove_node_name: str
):
    """
    Transforms the input and output of an ONNX model, renaming the input,
    removing all outputs, and creating a new output chain with Transpose and
    Identity nodes. Also removes a specified node from the model.

    Args:
        model_path (str): Path to the input ONNX model.
        output_path (str): Path to save the modified ONNX model.
        new_input_name (str): New name for the input tensor.
        new_output_names (List[str]): List of new names for the output tensors.
        internal_tensor_names (List[str]): List of internal tensor names to be
            used in the new output chain.
        remove_node_name (str): Name of the node to be removed from the model.
    """
    model = onnx.load(model_path)

    # Step 1: Rename input.
    old_input_name = model.graph.input[0].name
    model.graph.input[0].name = new_input_name
    for node in model.graph.node:
        node.input[:] = [
            new_input_name if i == old_input_name else i for i in node.input
        ]

    # Step 2: Remove all outputs.
    del model.graph.output[:]

    # Step 3: Build new output chain: Transpose → Identity → output.
    transpose_axes = [
        [0, 2, 1],  # Same for both outputs.
        [0, 2, 1],  # Same for both outputs.
    ]
    output_shapes = [
        [1, 8400,  4],  # For output_bboxes, for example
        [1, 8400, 34],  # For output_classes, for example
    ]

    new_nodes = []
    for i, (new_name, internal_name) in enumerate(
        zip(new_output_names, internal_tensor_names)
    ):
        transpose_name = f"{new_name}_transposed"

        # Transpose node.
        transpose_node = helper.make_node(
            "Transpose",
            inputs=[internal_name],
            outputs=[transpose_name],
            perm=transpose_axes[i],
            name=f"Transpose_{new_name}"
        )

        # Identity node to assign output name.
        identity_node = helper.make_node(
            "Identity",
            inputs=[transpose_name],
            outputs=[new_name],
            name=f"Identity_{new_name}"
        )

        # Output metadata.
        output_info = helper.make_tensor_value_info(
            new_name,
            TensorProto.FLOAT,
            output_shapes[i]
        )

        # Append to graph.
        new_nodes.extend([transpose_node, identity_node])
        model.graph.output.append(output_info)

    # Step 4: Remove the Concat node by name.
    original_nodes = [
        node for node in model.graph.node if node.name != remove_node_name
    ]

    # Step 5: Replace node list with cleaned + new output nodes.
    model.graph.ClearField("node")
    model.graph.node.extend(original_nodes + new_nodes)

    # Save model.
    onnx.save(model, output_path)

## Downloading the hagRID YOLO 11 Model Weights

The following code cell downloads the pretrained YOLO 11 model weights from Hugging Face and saves them to the `/models` directory. These weights are required for loading the hagRID-pretrained YOLO model and exporting it to ONNX format in subsequent steps. Make sure the download completes successfully before proceeding.

In [4]:
!wget https://huggingface.co/testdummyvt/hagRIDv2_gesture_det_models/resolve/main/yolo11n_10GB/train/weights/best.pt -q --show-progress -O /models/best.pt

/models/best.pt     100%[===================>]  15.34M  13.1MB/s    in 1.2s    


## Exporting and Transforming the hagRID YOLO 11 Model

The following code cell performs several key steps to prepare the hagRID-pretrained YOLO 11 model for downstream use:

- **Load the YOLO 11 model** using the downloaded weights.
- **Export the model to ONNX format** with the appropriate opset version.
- **Load the exported ONNX model** for further processing.
- **Transform the ONNX model** by:
    - Renaming the input and outputs to standardized names.
    - Creating new output chains for bounding boxes and class predictions.
    - Removing an unnecessary node from the computation graph.
- **Clean up** by deleting the original `.pt` and intermediate ONNX files to conserve disk space.

This process ensures the exported ONNX model is optimized and ready for integration into other applications or inference pipelines.

In [5]:
# Load the model.
model = YOLO("/models/best.pt")

# Export to ONNX.
model.export(format="onnx", opset=11)

# Transform the model by renaming inputs, outputs, and pruning a node.
transform_io_and_prune_node(
    model_path="/models/best.onnx",
    output_path="/models/yolo_hagRID.onnx",
    new_input_name="input",
    new_output_names=[
        "output_bboxes",
        "output_classes"
    ],
    internal_tensor_names=[
        "/model.23/Mul_2_output_0",
        "/model.23/Sigmoid_output_0"
    ],
    remove_node_name="/model.23/Concat_5"
)

# Validate the ONNX model.
model = onnx.load("/models/yolo_hagRID.onnx")
onnx.checker.check_model(model)
print("Model is valid!")

# Delete the original .pt and ONNX models to save space.
os.remove("/models/best.pt")
os.remove("/models/best.onnx")

Ultralytics 8.3.146 🚀 Python-3.10.16 torch-2.7.0+cu126 CPU (13th Gen Intel Core(TM) i9-13950HX)
YOLO11n summary (fused): 100 layers, 2,588,782 parameters, 0 gradients, 6.3 GFLOPs

PyTorch: starting from '/models/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 38, 8400) (15.3 MB)

ONNX: starting export with onnx 1.15.0 opset 11...
ONNX: slimming with onnxslim 0.1.54...
ONNX: export success ✅ 0.6s, saved as '/models/best.onnx' (10.1 MB)

Export complete (0.8s)
Results saved to /models
Predict:         yolo predict task=detect model=/models/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/models/best.onnx imgsz=640 data=/mnt/batch/tasks/shared/LS_root/mounts/clusters/t4-two/data/hagRIDv2_512px_10GB/yolo_format/data.yaml  
Visualize:       https://netron.app
Model is valid!


## Notebook Complete

You have now successfully prepared and exported both the COCO-pretrained YOLO-NAS S model and the hagRID-pretrained YOLO 11 model to ONNX format. These models are now ready for use in downstream tasks, such as inference, benchmarking, or integration into deployment pipelines.

If you encounter any issues or need to re-run the setup, simply restart the notebook and execute all cells in order.

For further information, refer to the project documentation or explore the exported ONNX models in the `/models` directory.

---
**Next Steps:**
- Use the exported ONNX models in your inference or deployment workflows.
- Explore other notebooks in this project for additional model processing and evaluation tasks.